In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/242B_final_project') # customize this line to your working directory

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/242B_final_project/np_save/"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import resize
import seaborn as sns
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn import functional as F
from torchvision.models import resnet50

import PIL.Image
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
import random

In [ ]:
train_matched_pairs = np.load(f'{path}train_matched_pairs.npy', allow_pickle=True)
train_matched_labels = np.load(f'{path}train_matched_labels.npy', allow_pickle=True)
test_matched_pairs = np.load(f'{path}test_matched_pairs.npy', allow_pickle=True)
test_matched_labels = np.load(f'{path}test_matched_labels.npy', allow_pickle=True)

train_mismatched_pairs = np.load(f'{path}train_mismatched_pairs.npy', allow_pickle=True)
train_mismatched_labels = np.load(f'{path}train_mismatched_labels.npy', allow_pickle=True)
test_mismatched_pairs = np.load(f'{path}test_mismatched_pairs.npy', allow_pickle=True)
test_mismatched_labels = np.load(f'{path}test_mismatched_labels.npy', allow_pickle=True)

# Data Loader


In [ ]:
class SiameseDataset(Dataset):
    def __init__(self, matched_pairs, matched_labels, mismatched_pairs, mismatched_labels):
        self.pairs = np.concatenate((matched_pairs, mismatched_pairs), axis=0)
        self.labels = np.concatenate((matched_labels, mismatched_labels), axis=0)

        # Shuffle the data
        indices = np.arange(len(self.pairs))
        random.shuffle(indices)
        self.pairs = self.pairs[indices]
        self.labels = self.labels[indices]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img1 = self.pairs[idx][0]
        img2 = self.pairs[idx][1]
        label = self.labels[idx]

        return img1, img2, label

In [ ]:
train_dataset = SiameseDataset(train_matched_pairs, train_matched_labels,
                                train_mismatched_pairs, train_mismatched_labels)

test_dataset = SiameseDataset(test_matched_pairs, test_matched_labels,
                               test_mismatched_pairs, test_mismatched_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Function

In [ ]:
def calculate_accuracy(data_loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            img1, img2, labels = data
            img1, img2 = img1.to(device), img2.to(device)
            labels = labels.to(device)
            labels = labels.squeeze()
            # print(f'labels: {labels}')
            outputs = model(img1, img2).squeeze()
            # print(f'Probability: {outputs}')
            predicted = (outputs > 0.5).float()  # Ensures it's a float tensor
            # print(f'Predicted: {predicted}')
            correct += (predicted == labels.float()).sum().item()  # Cast labels to float for comparison
            # print(f'Correct: {correct}')
            total += labels.size(0)
            # print(f'Total: {total}')
    return 100 * correct / total

In [ ]:
import time
from torch.utils.data import DataLoader

def train_model(model, criterion, num_epochs, optimizer, train_loader, test_loader): #test_loader
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()  # Set the model to training mode

    # Logs for keeping track of loss and accuracy
    train_loss_log = []
    train_acc_log = []
    test_acc_log = []

    tic = time.time()

    for epoch in range(num_epochs):
        total_train_loss = 0
        model.train()  # Ensure model is in training mode

        for data in train_loader:
            img1, img2, labels = data
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(img1, img2)  # Adjusted to use the updated forward method
            loss = criterion(outputs, labels.float().view(-1, 1))

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        # Calculate average loss and accuracy for the epoch
        avg_train_loss = total_train_loss / len(train_loader)
        train_accuracy = calculate_accuracy(train_loader, model, device)
        test_accuracy = calculate_accuracy(test_loader, model, device)

        train_loss_log.append(avg_train_loss)
        train_acc_log.append(train_accuracy)
        test_acc_log.append(test_accuracy)

        # Print statistics
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')
        # print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')

    toc = time.time()
    print("Elapsed Time : {:7.2f}".format(toc-tic))

    return train_loss_log, train_acc_log , test_acc_log


# Model 4 - VGG16

In [ ]:
class SiameseVGG16(nn.Module):
    def __init__(self):
        super(SiameseVGG16, self).__init__()
        # Load a pre-trained VGG16 model
        self.vgg16 = models.vgg16(pretrained=True)

        # Remove the classifier part and retain only the features
        self.features = self.vgg16.features

        # Freeze the feature extraction part to not train it again
        for param in self.features.parameters():
            param.requires_grad = False

        # Adding an adaptive pooling layer before the classifier
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))  # Outputs a fixed size regardless of input size

        # Adding a new classifier
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward_once(self, x):
        # Forward pass through VGG16 features
        x = self.features(x)
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)  # Flatten the features
        return x

    def forward(self, input1, input2):
        # Generate features from both inputs
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        # Concatenate or subtract features (depending on your approach)
        combined_features = torch.abs(output1 - output2)

        # Pass through the classifier to get the final output
        similarity = self.classifier(combined_features)
        return similarity

# Example of initializing the model
model = SiameseVGG16()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 10
criterion = nn.BCELoss()  # Ensure the label and output size match
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_loss_log, train_acc_log, test_acc_log= train_model(model, criterion, num_epochs, optimizer, train_loader, test_loader)

Epoch 1/10, Training Loss: 1.0462, Training Accuracy: 75.55%, Test Accuracy: 58.50%
Epoch 2/10, Training Loss: 0.5659, Training Accuracy: 88.59%, Test Accuracy: 62.90%
Epoch 3/10, Training Loss: 0.4156, Training Accuracy: 94.73%, Test Accuracy: 69.40%
Epoch 4/10, Training Loss: 0.3019, Training Accuracy: 97.45%, Test Accuracy: 68.50%
Epoch 5/10, Training Loss: 0.2302, Training Accuracy: 98.05%, Test Accuracy: 65.10%
Epoch 6/10, Training Loss: 0.1674, Training Accuracy: 99.14%, Test Accuracy: 68.80%
Epoch 7/10, Training Loss: 0.1229, Training Accuracy: 99.55%, Test Accuracy: 69.00%
Epoch 8/10, Training Loss: 0.1359, Training Accuracy: 98.91%, Test Accuracy: 66.20%
Epoch 9/10, Training Loss: 0.0924, Training Accuracy: 99.64%, Test Accuracy: 68.20%
Epoch 10/10, Training Loss: 0.0861, Training Accuracy: 99.77%, Test Accuracy: 67.80%
Elapsed Time :  524.31
